In [1]:
import pandas as pd 
import numpy as np
import dask.dataframe as dd
import dask
from dask.distributed import Client

dask.config.set({'dataframe.query-planning':True})
client = Client()

n=20
pd.set_option('display.max_rows', n)

In [2]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 3.74 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46185,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 3.74 GiB
Comm: tcp://127.0.0.1:42591,Total threads: 2
Dashboard: http://127.0.0.1:37055/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:44427,


In [3]:
#loading data 

path = '/mnt/c/Users/Utente/Downloads/Z0.02_a3_cM70/output_0.csv'
#path = ''
df = dd.read_csv(path, dtype={'BEvent': float})

### Selecting systems

In [4]:
#### Wolf Rayet - Black hole system
cond_1 = ((df.PhaseBSE_0==8) | (df.PhaseBSE_0==7)) & (df.PhaseBSE_1==14.0) 
cond_2 = ((df.PhaseBSE_1==8) | (df.PhaseBSE_1==7)) & (df.PhaseBSE_0==14.0)
df_WRBH = df.loc[ cond_1 | cond_2 ]

data_WRBH = df_WRBH[['ID', 'Mass_0', 'Mass_1', 'Semimajor', 'Eccentricity']].drop_duplicates('ID') #selecting only 1st row of each ID

id_WRBH = data_WRBH['ID'] #taking only the IDs

In [5]:
#### GW BBH merge (Binary Black Holes that merge via Gravitational Waves Emission in less than 14 bilions years)
cond_1 = (df.PhaseBSE_0 == 14.0) & (df.PhaseBSE_1 == 14.0)
cond_2 = (df.GWtime + df.BWorldtime) < int(14e+03)
id_GW = df.loc[cond_1 & cond_2]['ID'].drop_duplicates()

In [6]:
id_WRBH.compute()
id_GW.compute()

id_both = np.intersect1d(id_WRBH, id_GW) #ID of WR-BH systems that evolve in a BH-BH systems that merge via GW emission

2024-06-04 14:13:20,785 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 685.86 MiB -- Worker memory limit: 0.93 GiB
2024-06-04 14:13:43,565 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 653.17 MiB -- Worker memory limit: 0.93 GiB
2024-06-04 14:14:02,248 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

2024-06-04 14:37:05,870 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:42591' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('drop-duplicates-combine-3a399591379af5083566a8ea73bd9423', 39, 1), ('drop-duplicates-chunk-3a399591379af5083566a8ea73bd9423-bc39fe9aa197ddb6c180283a917e0456', 266), ('drop-duplicates-combine-3a399591379af5083566a8ea73bd9423', 38, 1)} (stimulus_id='handle-worker-cleanup-1717504625.8330348')
2024-06-04 14:37:06,440 - distributed.nanny - WARNING - Restarting worker
2024-06-04 14:37:46,391 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:41249' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('drop-duplicates-chunk-3a399591379af5083566a8ea73bd9423-bc39fe9aa197ddb6c180283a917e0456', 247), ('drop-duplicates-combine-3a399591379af5083566a8ea73bd9423', 1, 3)} (stimulus_id='handle-worker-cleanup-1717504666.3851025')
2024-06-04 14:37:46,788 - dist

In [ ]:
'''
print('Binary systems of a Wolf-Rayet and a Black Hole: ', len(id_WRBH))
print('Binary systems that merge via GW: ', len(id_GW))
print('Binary systems that evolve from a WR-BH and merge via GW: ', len(id_both))
'''

Binary systems of a Wolf-Rayet and a Black Hole:  130479
Binary systems that merge via GW:  340
Binary systems that evolve from a WR-BH and merge via GW:  306


### Mass transfer events

In [7]:
# counting the number of MT stable events (Roche Lobe)
n_stable = df_WRBH.loc[df_WRBH['BEvent']==5, ['ID','BEvent']].groupby('ID').count().reset_index() #reset index serve per avere ID come colonna e non come indice

In [8]:
# counting the number of MT unstable events (Common Envelope)
n_unstable = df_WRBH.loc[(df_WRBH['BEvent']==7) | (df_WRBH['BEvent']==11), ['ID','BEvent']].groupby('ID').count().reset_index()

In [9]:
#merging the two tables
mass_transfer = n_stable.merge(n_unstable, how='outer', on='ID')
mass_transfer = mass_transfer.rename(columns={'BEvent_x': 'MTEvents_stable', 'BEvent_y': 'MTEvents_unstable'})

#sustituting NA values with zeros
mass_transfer['MTEvents_unstable'] = mass_transfer['MTEvents_unstable'].fillna(0).astype(int)
mass_transfer['MTEvents_stable'] = mass_transfer['MTEvents_stable'].fillna(0).astype(int)

In [10]:
#merge stable-unstable table with the table containing parameters of the binary system
data = mass_transfer.merge(data_WRBH, how='inner', on='ID')

#adding column which says if the system merge
col = data['ID'].isin(id_both).astype(int)
data = data.assign(Merge=col)

In [11]:
#saving the output
data.to_csv('./out.csv', index=False)  

2024-06-04 15:42:37,590 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 670.80 MiB -- Worker memory limit: 0.93 GiB
2024-06-04 15:42:38,102 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 709.03 MiB -- Worker memory limit: 0.93 GiB
2024-06-04 15:42:41,979 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
2024-06-04 15:44:01,610 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/process.py", line 202, in _run
    target(*args, **kwargs)
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/nanny.py", line 1019, in _run
    asyncio_run(run(), loop_factory=get_loop_factory())
  File "/home/gaia/.local/lib/python3.10/site-packages/distributed/compatibility.py", line 236, in asyncio_run
    return loop.run_until_complete(main)
  File "/usr/lib/python3.10/asyncio/base_events.py", line 636, in run_until_complete
    self.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
   